In [2]:
import gzip
import json

import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [58]:
fl=gzip.open('goemotions.json.gz')
res=json.load(fl)
#test
print(res[2])

['Man I love reddit.', 'love', 'positive']


In [42]:
df=pd.DataFrame(res,columns=["post","emotion","sentiment"])
emotions_and_sentiments=df.drop(columns=['post'])

In [52]:
posts=df.drop(columns=['emotion','sentiment'])
numpy_posts=posts.to_numpy().flatten()
numpy_emotions_and_sentiments=emotions_and_sentiments.to_numpy()

In [60]:
numpy_emotions_and_sentiments[2][1]

'positive'

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(numpy_posts,numpy_emotions_and_sentiments,test_size=0.2,train_size=0.8)

In [69]:
y_test[:,1]

array(['neutral', 'neutral', 'positive', ..., 'negative', 'neutral',
       'negative'], dtype=object)

In [6]:
import gensim.downloader as api
embedding=api.load("word2vec-google-news-300")

In [13]:
# import gensim
# from gensim import models
# from gensim.models import Word2Vec
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [70]:
print(X_train)

['please don’t make me have to bring up fire emblem in trader joe’s again i can never go back'
 '😂 sometimes I love old people. Not always. But sometimes! I ant wait to be an old ass troll saying shit like this.'
 "Maybe or my reptilian DNA hasn't manifested yet" ...
 "I'm sorry [NAME]. I won't do it again."
 "I didn't know macs were THAT limited. /s"
 "If they didn't want the wheels to be stolen, they shouldn't have made then so easily stealable!"]


In [7]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

In [76]:
def countTokens(X):
 token_word = []
 for words in X:
  token_word.extend(word_tokenize(words))
 return len(token_word)


In [81]:
X_train_tokens_count=countTokens(X_train)

In [82]:
X_test_tokens_count=countTokens(X_test)

In [87]:
def getEmbeddings(X):
 posts_embedding=[]
 count=0
 for post in X:
  words=word_tokenize(post)
  word_embedding=[]
  for word in words:
    try:
     temp=embedding[word]
     word_embedding.append(temp)
    except KeyError:
     count+=1
     pass
  accumumlate=[0]*300
  accumumlate=np.array(accumumlate)
  for embedding_vec in word_embedding:
          accumumlate=embedding_vec+accumumlate
  if len(word_embedding)!=0:
     accumumlate=accumumlate/len(word_embedding)
     posts_embedding.append(numpy.copy(accumumlate))
 return posts_embedding,count






In [88]:
X_train_embeddings,X_train_misses=getEmbeddings(X_train)

In [89]:
X_test_embeddings,X_test_misses=getEmbeddings(X_test)

In [92]:
hit_rate_train=100-((X_train_misses/X_train_tokens_count)*100)
print(hit_rate_train)

77.45391136930904


In [93]:
hit_rate_test=100-((X_test_misses/X_test_tokens_count)*100)
print(hit_rate_test)

77.43750557266085


In [91]:
X_train_misses

476850